In [36]:
import pandas as pd
import numpy as np

from influxdb_client import InfluxDBClient, Point, Dialect

import re
import time
import os
import datetime

import warnings
from influxdb_client.client.warnings import MissingPivotFunction

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors

import pandasql as ps
import sqlite3

import csv

import warnings 
from influxdb_client.client.warnings import MissingPivotFunction
warnings.simplefilter("ignore", MissingPivotFunction)

import gc

pd.set_option('display.max_rows', 500)

In [2]:
data = pd.read_csv('./experiments_filtered.csv', header=0,  index_col=0)
experiments = pd.DataFrame(data)

experiments.head(10)

,experiment,originalStart,originalEnd,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,start,end,startUnix,endUnix
3,390,2023-10-04 18:52:03.080773000,2023-10-04 19:22:03.086651800,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-10-04 18:59:33.080773000,2023-10-04 19:14:33.086651800,1696445973,1696446873
4,391,2023-10-04 19:25:37.153339900,2023-10-04 19:55:37.155585600,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-10-04 19:33:07.153339900,2023-10-04 19:48:07.155585600,1696447987,1696448887
6,393,2023-10-04 20:32:47.309861600,2023-10-04 21:02:47.314917800,unl,informed,8,6,12,4,8,2,0.25,3.0,2023-10-04 20:40:17.309861600,2023-10-04 20:55:17.314917800,1696452017,1696452917
7,394,2023-10-04 21:06:22.775563400,2023-10-04 21:36:22.778880000,unl,informed,8,6,12,4,8,2,0.25,3.0,2023-10-04 21:13:52.775563400,2023-10-04 21:28:52.778880000,1696454032,1696454932
9,396,2023-10-04 22:13:33.814670400,2023-10-04 22:43:33.817409400,unl,informed0,16,8,20,16,8,4,0.50,1.0,2023-10-04 22:21:03.814670400,2023-10-04 22:36:03.817409400,1696458063,1696458963
10,397,2023-10-04 22:47:09.383711000,2023-10-04 23:17:09.386844200,unl,informed0,16,8,20,16,8,4,0.50,1.0,2023-10-04 22:54:39.383711000,2023-10-04 23:09:39.386844200,1696460079,1696460979
15,404,2023-10-05 02:42:12.104487300,2023-10-05 03:12:12.156487200,unl,informed2,16,8,20,16,8,4,0.50,0.5,2023-10-05 02:49:42.104487300,2023-10-05 03:04:42.156487200,1696474182,1696475082
16,405,2023-10-05 03:15:44.641811200,2023-10-05 03:45:44.644068700,unl,informed3,6,4,8,2,2,2,0.25,1.0,2023-10-05 03:23:14.641811200,2023-10-05 03:38:14.644068700,1696476194,1696477094
17,406,2023-10-05 03:49:18.120429500,2023-10-05 04:19:18.122983600,unl,informed3,6,4,8,2,2,2,0.25,1.0,2023-10-05 03:56:48.120429500,2023-10-05 04:11:48.122983600,1696478208,1696479108
21,412,2023-10-05 07:10:41.394283100,2023-10-05 07:40:41.433055700,unl,informed5,6,4,8,2,2,4,0.25,0.5,2023-10-05 07:18:11.394283100,2023-10-05 07:33:11.433055700,1696490291,1696491191


In [3]:
data = pd.read_csv('./consensus.csv', header=0,  index_col=0)
parameters = pd.DataFrame(data)

parameters.head(10)

,identifier,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,n_nodes,consensus
0,0,unl,8,6,12,4,8,2,0.25,1.0,24,3.022872
1,3,unl,8,6,12,4,8,2,0.25,3.0,24,3.026471
2,6,unl,16,8,20,16,8,4,0.50,1.0,24,3.027537
3,12,unl,16,8,20,16,8,4,0.50,0.5,24,3.073917
4,13,unl,6,4,8,2,2,2,0.25,1.0,24,3.038808
5,18,unl,6,4,8,2,2,4,0.25,0.5,24,3.014395
6,21,unl,20,16,24,8,16,8,0.50,1.0,24,3.029904
7,23,unl,16,8,20,4,8,2,0.50,1.0,24,3.021019
8,27,unl,16,8,20,8,8,4,0.25,1.0,24,3.025704
9,29,unl,16,8,20,4,8,2,0.25,1.0,24,3.044116


In [6]:
merged = experiments.merge(parameters, on=['topology','d','dhi','dlo','dlazy','dscore','dout','gossipFactor','interval'])
merged = merged.drop(columns=['consensus', 'n_nodes', 'originalStart', 'originalEnd', 'start', 'end', 'startUnix', 'endUnix']).rename(columns={'parameter' :  'reference'})

merged.head(10)

,experiment,topology,reference,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,identifier
0,390,unl,reference,8,6,12,4,8,2,0.25,1.0,0
1,391,unl,reference,8,6,12,4,8,2,0.25,1.0,0
2,393,unl,informed,8,6,12,4,8,2,0.25,3.0,3
3,394,unl,informed,8,6,12,4,8,2,0.25,3.0,3
4,396,unl,informed0,16,8,20,16,8,4,0.50,1.0,6
5,397,unl,informed0,16,8,20,16,8,4,0.50,1.0,6
6,404,unl,informed2,16,8,20,16,8,4,0.50,0.5,12
7,405,unl,informed3,6,4,8,2,2,2,0.25,1.0,13
8,406,unl,informed3,6,4,8,2,2,2,0.25,1.0,13
9,412,unl,informed5,6,4,8,2,2,4,0.25,0.5,18


In [13]:
#Selected first of each one
merged = merged.sort_values(by=['identifier', 'experiment'], ascending=False)
final = merged.drop_duplicates(subset=['identifier'])

final.head(100)

,experiment,topology,reference,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,identifier
48,499,general,random7,10,8,16,4,8,4,0.25,1.00,85
46,497,general,random6,16,8,20,6,8,2,0.25,1.00,83
44,494,general,random5,8,6,16,4,8,2,0.25,0.25,80
42,484,general,random2,8,6,16,4,8,2,0.25,1.00,75
40,482,general,random1,16,8,20,4,8,2,0.25,1.00,72
38,478,general,informed8,16,8,20,8,8,4,0.25,1.00,69
36,476,general,informed7,16,8,20,4,8,2,0.50,1.00,67
34,473,general,informed6,20,16,24,8,16,8,0.50,1.00,63
32,469,general,informed5,6,4,8,2,2,4,0.25,0.50,61
31,463,general,informed3,6,4,8,2,2,2,0.25,1.00,56


In [22]:
message = pd.DataFrame()
execs = experiments.drop(columns=['originalStart', 'originalEnd'])
# final=final.loc[final["identifier"] == 1]

for index, row in final.iterrows():

    if  os.path.isfile('./ihave_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('./ihave_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        ihave = pd.DataFrame(data)
    else:
         ihave = pd.DataFrame()

    if  os.path.isfile('./iwant_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('./iwant_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        iwant = pd.DataFrame(data)
    else:
        iwant = pd.DataFrame()

    data = pd.read_csv('./mess_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
    mess = pd.DataFrame(data)

    ihave['type'] = 17
    iwant['type'] = 16

    gossip = pd.concat([ihave, iwant])
    gossip = gossip.drop(columns=['_measurement'])
    gossip['identifier'] = row['identifier']

    mess = mess.drop(columns=['start', 'end'])
    mess['identifier'] = row['identifier']

    message = pd.concat([message,mess,gossip])
        
    # del query_mess
    # del deliverMessage
    del ihave
    del iwant
    del gossip
    del mess
    gc.collect()

# message.head(10)
message.head(100)

,_time,type,identifier,experiment
0,2023-10-07 17:51:03.063304+00:00,3.0,85,499.0
1,2023-10-07 17:51:03.063779+00:00,3.0,85,499.0
2,2023-10-07 17:51:03.064200+00:00,2.0,85,499.0
3,2023-10-07 17:51:03.064207+00:00,2.0,85,499.0
4,2023-10-07 17:51:03.064238+00:00,2.0,85,499.0
...,...,...,...,...
95,2023-10-07 17:51:03.065961+00:00,2.0,85,499.0
96,2023-10-07 17:51:03.065997+00:00,2.0,85,499.0
97,2023-10-07 17:51:03.066045+00:00,2.0,85,499.0
98,2023-10-07 17:51:03.066053+00:00,2.0,85,499.0


In [23]:
print(message)

                               _time  type  identifier  experiment
0   2023-10-07 17:51:03.063304+00:00   3.0          85       499.0
1   2023-10-07 17:51:03.063779+00:00   3.0          85       499.0
2   2023-10-07 17:51:03.064200+00:00   2.0          85       499.0
3   2023-10-07 17:51:03.064207+00:00   2.0          85       499.0
4   2023-10-07 17:51:03.064238+00:00   2.0          85       499.0
..                               ...   ...         ...         ...
12  2023-10-04 19:44:44.061406+00:00  16.0           0       391.0
13  2023-10-04 19:45:47.232855+00:00  16.0           0       391.0
14  2023-10-04 19:52:44.235604+00:00  16.0           0       391.0
15  2023-10-04 19:52:44.236336+00:00  16.0           0       391.0
16  2023-10-04 19:52:44.236490+00:00  16.0           0       391.0

[42728506 rows x 4 columns]


In [30]:
dfNoIndex = message.reset_index()
dfNoIndex["_time"] = pd.to_datetime(dfNoIndex["_time"],format="mixed", infer_datetime_format=True)
# dfNoIndex.head(10)

by_time = dfNoIndex.groupby([dfNoIndex['identifier'],dfNoIndex['experiment'],dfNoIndex['type'],pd.Grouper(key="_time", freq='10s')]).count().reset_index()
dfAggTime1 = by_time.rename(columns={'index': "count"})

dfAggTime1.tail(10)

/tmp/ipykernel_203569/3880853263.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dfNoIndex["_time"] = pd.to_datetime(dfNoIndex["_time"],format="mixed", infer_datetime_format=True)


,identifier,experiment,type,_time,count
12255,85,499.0,17.0,2023-10-07 18:19:30+00:00,120
12256,85,499.0,17.0,2023-10-07 18:19:40+00:00,120
12257,85,499.0,17.0,2023-10-07 18:19:50+00:00,120
12258,85,499.0,17.0,2023-10-07 18:20:00+00:00,120
12259,85,499.0,17.0,2023-10-07 18:20:10+00:00,120
12260,85,499.0,17.0,2023-10-07 18:20:20+00:00,120
12261,85,499.0,17.0,2023-10-07 18:20:30+00:00,120
12262,85,499.0,17.0,2023-10-07 18:20:40+00:00,120
12263,85,499.0,17.0,2023-10-07 18:20:50+00:00,120
12264,85,499.0,17.0,2023-10-07 18:21:00+00:00,24


In [25]:

dfAggTime1.to_csv('./timeSeries10s.csv')

In [31]:
by_time10 = dfNoIndex.groupby([dfNoIndex['identifier'],dfNoIndex['experiment'],dfNoIndex['type'],pd.Grouper(key="_time", freq='1s')]).count().reset_index()
dfAggTime10 = by_time10.rename(columns={'index': "count"})

# dfAggTime.to_csv('./timeSeries.csv')
dfAggTime10.tail(10)

,identifier,experiment,type,_time,count
88152,85,499.0,17.0,2023-10-07 18:20:52+00:00,12
88153,85,499.0,17.0,2023-10-07 18:20:53+00:00,12
88154,85,499.0,17.0,2023-10-07 18:20:54+00:00,12
88155,85,499.0,17.0,2023-10-07 18:20:55+00:00,12
88156,85,499.0,17.0,2023-10-07 18:20:56+00:00,12
88157,85,499.0,17.0,2023-10-07 18:20:57+00:00,12
88158,85,499.0,17.0,2023-10-07 18:20:58+00:00,12
88159,85,499.0,17.0,2023-10-07 18:20:59+00:00,12
88160,85,499.0,17.0,2023-10-07 18:21:00+00:00,12
88161,85,499.0,17.0,2023-10-07 18:21:01+00:00,12


In [37]:
#Verify numbers

verify1 = dfAggTime1.drop(columns=['_time'])
verify1 = verify1.groupby(['experiment', 'identifier', 'type']).agg('sum')
verify1.head(100)

count
experiment identifier type         
391.0      0          2.0   1681977
                      3.0    242460
                      11.0     1537
                      12.0      125
                      16.0       17
                      17.0   109756
394.0      3          2.0   1843504
                      3.0    164486
                      11.0     2137
                      16.0        1
                      17.0     4868
397.0      6          2.0   1428254
                      3.0    124371
                      11.0     2138
                      17.0      792
404.0      12         2.0    762906
                      3.0    156136
                      11.0     1387
                      12.0      135
                      16.0       12
                      17.0    71454
406.0      13         2.0   1778309
                      3.0    351875
                      11.0     1469
                      12.0      308
                      16.0       46
                      17.0   141786
413.0      18         2.0    870750
                      3.0    151214
                      11.0     2136
                      16.0        2
                      17.0     4869
416.0      21         2.0   1998816
                      3.0    205431
                      11.0     2134
                      17.0     9774
419.0      23         2.0   2013329
                      3.0    198723
                      11.0     1819
                      16.0       14
                      17.0     8629
421.0      27         2.0   1957411
                      3.0    217007
                      11.0     2138
                      16.0        1
                      17.0     9194
425.0      29         2.0   1008492
                      3.0    108908
                      11.0     1777
                      12.0       14
                      16.0       23
                      17.0   185785
426.0      32         2.0   1440218
                      3.0    137453
                      11.0     1739
                      12.0       28
                      16.0       26
                      17.0   133297
437.0      36         2.0   1268845
                      3.0    205555
                      11.0     1888
                      16.0       11
                      17.0    79433
439.0      39         2.0   1541024
                      3.0    205727
                      11.0     1815
                      12.0        1
                      16.0       20
                      17.0   105559
442.0      42         2.0   1708560
                      3.0    220485
                      11.0     1734
                      16.0       31
                      17.0   179182
448.0      43         2.0   1136304
                      3.0    162321
                      11.0      297
                      12.0       33
                      16.0       18
                      17.0   145200
451.0      47         2.0   1846117
                      3.0    142697
                      11.0      432
                      12.0       22
                      16.0       40
                      17.0   230620
454.0      49         2.0   3116919
                      3.0    200319
                      11.0      464
                      12.0       23
                      16.0       20
                      17.0    90616
461.0      54         2.0    384026
                      3.0     80689
                      11.0      332
                      12.0      128
                      16.0        8
                      17.0    40464
463.0      56         2.0    348588
                      3.0     67463

In [38]:
#Verify numbers

verify10 = dfAggTime10.drop(columns=['_time'])
verify10 = verify1.groupby(['experiment', 'identifier', 'type']).agg('sum')
verify10.head(100)

count
experiment identifier type         
391.0      0          2.0   1681977
                      3.0    242460
                      11.0     1537
                      12.0      125
                      16.0       17
                      17.0   109756
394.0      3          2.0   1843504
                      3.0    164486
                      11.0     2137
                      16.0        1
                      17.0     4868
397.0      6          2.0   1428254
                      3.0    124371
                      11.0     2138
                      17.0      792
404.0      12         2.0    762906
                      3.0    156136
                      11.0     1387
                      12.0      135
                      16.0       12
                      17.0    71454
406.0      13         2.0   1778309
                      3.0    351875
                      11.0     1469
                      12.0      308
                      16.0       46
                      17.0   141786
413.0      18         2.0    870750
                      3.0    151214
                      11.0     2136
                      16.0        2
                      17.0     4869
416.0      21         2.0   1998816
                      3.0    205431
                      11.0     2134
                      17.0     9774
419.0      23         2.0   2013329
                      3.0    198723
                      11.0     1819
                      16.0       14
                      17.0     8629
421.0      27         2.0   1957411
                      3.0    217007
                      11.0     2138
                      16.0        1
                      17.0     9194
425.0      29         2.0   1008492
                      3.0    108908
                      11.0     1777
                      12.0       14
                      16.0       23
                      17.0   185785
426.0      32         2.0   1440218
                      3.0    137453
                      11.0     1739
                      12.0       28
                      16.0       26
                      17.0   133297
437.0      36         2.0   1268845
                      3.0    205555
                      11.0     1888
                      16.0       11
                      17.0    79433
439.0      39         2.0   1541024
                      3.0    205727
                      11.0     1815
                      12.0        1
                      16.0       20
                      17.0   105559
442.0      42         2.0   1708560
                      3.0    220485
                      11.0     1734
                      16.0       31
                      17.0   179182
448.0      43         2.0   1136304
                      3.0    162321
                      11.0      297
                      12.0       33
                      16.0       18
                      17.0   145200
451.0      47         2.0   1846117
                      3.0    142697
                      11.0      432
                      12.0       22
                      16.0       40
                      17.0   230620
454.0      49         2.0   3116919
                      3.0    200319
                      11.0      464
                      12.0       23
                      16.0       20
                      17.0    90616
461.0      54         2.0    384026
                      3.0     80689
                      11.0      332
                      12.0      128
                      16.0        8
                      17.0    40464
463.0      56         2.0    348588
                      3.0     67463

In [41]:
verify = verify1.merge(verify10, on=['experiment', 'identifier', 'type'])

verify.head(100)

count_x  count_y
experiment identifier type                  
391.0      0          2.0   1681977  1681977
                      3.0    242460   242460
                      11.0     1537     1537
                      12.0      125      125
                      16.0       17       17
                      17.0   109756   109756
394.0      3          2.0   1843504  1843504
                      3.0    164486   164486
                      11.0     2137     2137
                      16.0        1        1
                      17.0     4868     4868
397.0      6          2.0   1428254  1428254
                      3.0    124371   124371
                      11.0     2138     2138
                      17.0      792      792
404.0      12         2.0    762906   762906
                      3.0    156136   156136
                      11.0     1387     1387
                      12.0      135      135
                      16.0       12       12
                      17.0    71454    71454
406.0      13         2.0   1778309  1778309
                      3.0    351875   351875
                      11.0     1469     1469
                      12.0      308      308
                      16.0       46       46
                      17.0   141786   141786
413.0      18         2.0    870750   870750
                      3.0    151214   151214
                      11.0     2136     2136
                      16.0        2        2
                      17.0     4869     4869
416.0      21         2.0   1998816  1998816
                      3.0    205431   205431
                      11.0     2134     2134
                      17.0     9774     9774
419.0      23         2.0   2013329  2013329
                      3.0    198723   198723
                      11.0     1819     1819
                      16.0       14       14
                      17.0     8629     8629
421.0      27         2.0   1957411  1957411
                      3.0    217007   217007
                      11.0     2138     2138
                      16.0        1        1
                      17.0     9194     9194
425.0      29         2.0   1008492  1008492
                      3.0    108908   108908
                      11.0     1777     1777
                      12.0       14       14
                      16.0       23       23
                      17.0   185785   185785
426.0      32         2.0   1440218  1440218
                      3.0    137453   137453
                      11.0     1739     1739
                      12.0       28       28
                      16.0       26       26
                      17.0   133297   133297
437.0      36         2.0   1268845  1268845
                      3.0    205555   205555
                      11.0     1888     1888
                      16.0       11       11
                      17.0    79433    79433
439.0      39         2.0   1541024  1541024
                      3.0    205727   205727
                      11.0     1815     1815
                      12.0        1        1
                      16.0       20       20
                      17.0   105559   105559
442.0      42         2.0   1708560  1708560
                      3.0    220485   220485
                      11.0     1734     1734
                      16.0       31       31
                      17.0   179182   179182
448.0      43         2.0   1136304  1136304
                      3.0    162321   162321
                      11.0      297      297
                      12.0       33       33
                      16.0       18       18
                      17.0   145200   145200
451.0      47         2.0   1846117  1846117
                      3.0    142697   142697
                      11.0      432      432
                      12.0       22       22
                      16.0       40       40
                      17.0   230620   230620
454.0      49         2.0   3116919  3116919
                      3

In [43]:
dfAggTime10.to_csv('./timeSeries10_verified.csv')
dfAggTime1.to_csv('./timeSeries1_verified.csv')